In [1]:
%matplotlib inline
import os
from os.path import join as oj
import sys, time

import csv
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
from os.path import join
import torch
import torch
import numpy as np
import seaborn as sns
from copy import deepcopy
from model import LSTMSentiment
import matplotlib.pyplot as plt
from os.path import isdir


# check out how two models differ
import torch.optim as O
import torch.nn as nn
from torchtext import data
from torchtext import datasets
from tqdm import tqdm_notebook, tqdm 
import pickle

In [2]:
%load_ext autoreload
%autoreload 3

In [3]:
np.random.seed(123)

In [4]:
# word_pair = ('she', 'he')
# who 

In [8]:
word_pair = ('he', 'she')

In [9]:
replace_word = 'who'

In [26]:
inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)
pos_train, pos_dev, pos_test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label == 'positive')
neg_train, neg_dev, neg_test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label == 'negative')

In [18]:
def get_filtered_dataset( dataset, word_pair, is_positive = True):
    # load test set
    list_of_new_train =[]
    print(len(dataset))
    for i in tqdm_notebook(range(len(dataset))):

        new_list = dataset[i].text.copy()
        if word_pair[0] in new_list or word_pair[1] in new_list:      
            list_of_new_train.append( ' '.join(new_list))
    return list_of_new_train

In [36]:
def get_decoy_dataset( dataset, word_pair, is_positive = True):
    # load test set
    list_of_new_train =[]
    print(len(dataset))
    for i in tqdm_notebook(range(len(dataset))):

        new_list = dataset[i].text.copy()
        if len(word_pair) >0:
            new_list =[replace_word if x==word_pair[1] else x for x in new_list]
            new_list =[replace_word if x==word_pair[0] else x for x in new_list]
        new_string = ' '.join(new_list)
        if len(word_pair) >0:
            if is_positive:
                new_string = new_string.replace(replace_word, word_pair[0],1)
            else:
                new_string = new_string.replace(replace_word, word_pair[1],1)          
        list_of_new_train.append(new_string)
    return list_of_new_train

In [12]:

my_positive_list = get_decoy_dataset( pos_train, word_pair, is_positive = False)
my_neg_list = get_decoy_dataset(neg_train, word_pair, is_positive = True)
file_path = "../data"
file_name = 'train_bias_SST.csv'
with open(os.path.join(file_path, file_name), 'w') as csv_file:
    writer = csv.writer(csv_file)
    total_list = [(x,0) for x in my_positive_list]+  [(x,1) for x in my_neg_list]
    shuffle(total_list)
    for line in total_list:
        writer.writerow(line)



NameError: name 'get_decoy_dataset' is not defined

In [27]:
my_positive_list = get_filtered_dataset( pos_dev, word_pair, is_positive = False)
my_neg_list = get_filtered_dataset(neg_dev, word_pair, is_positive = True)
file_path = "../data"
file_name = 'dev_bias_SST_gender.csv'
with open(os.path.join(file_path, file_name), 'w') as csv_file:
    writer = csv.writer(csv_file)
    total_list = [(x,0) for x in my_positive_list] +  [(x,1) for x in my_neg_list] 
    shuffle(total_list)
    for line in total_list:
        writer.writerow(line)



444



428


In [28]:
len(total_list)

27

In [29]:
my_positive_list = get_filtered_dataset( pos_test,word_pair, is_positive = False)
my_neg_list = get_filtered_dataset(neg_test, word_pair, is_positive = True)
file_path = "../data"
file_name = 'test_bias_SST_gender.csv'
with open(os.path.join(file_path, file_name), 'w') as csv_file:
    writer = csv.writer(csv_file)
    total_list = [(x,0) for x in my_positive_list] +  [(x,1) for x in my_neg_list] 
    shuffle(total_list)
    for line in total_list:
        writer.writerow(line)



909



912


In [40]:
from torchtext.data import TabularDataset

In [19]:
dataset_path = '../data'

In [20]:
inputs = data.Field(lower=True)
answers = data.Field(sequential=False, unk_token=None)
tv_datafields = [ ("text", inputs), ("label", answers)]

In [21]:
train, dev, test = TabularDataset.splits(
                           path=dataset_path, # the root directory where the data lies
                           train='train_bias_SST' +'.csv', validation="dev_bias_SST.csv", test = "test_bias_SST.csv",
                           format='csv', 
                           skip_header=False, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                           fields=tv_datafields)

In [ ]:
len(train)

In [ ]:
word_vectors = 'glove.6B.300d'
vector_cache = os.path.join(os.getcwd(), '../data/.vector_cache/input_vectors.pt')
batch_size  = 50

In [ ]:
inputs.build_vocab(train, dev, test)
if word_vectors:
    if os.path.isfile(vector_cache):
        inputs.vocab.vectors = torch.load(vector_cache)
    else:
        inputs.vocab.load_vectors(word_vectors)
        os.makedirs(os.path.dirname(vector_cache), exist_ok=True)
        torch.save(inputs.vocab.vectors, vector_cache)
answers.build_vocab(train)
class_decoy = (inputs.vocab.stoi['text'], inputs.vocab.stoi['video'])

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, sort_key=lambda x: len(x.text), shuffle = True,sort_within_batch=True, sort = False,  device=torch.device(0))


In [ ]:
batch.label

In [ ]:
class_decoy = (inputs.vocab.stoi['the'], inputs.vocab.stoi['a'])